In [1]:

from google.colab import files
import zipfile

# Upload zip files
uploaded = files.upload()

# Extract each uploaded zip to its own folder
for zip_file in uploaded.keys():
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(f"/content/{zip_file[:-4]}")

print("Files extracted.")


Saving box.zip to box.zip
Saving img.zip to img.zip
Files extracted.


In [3]:
get_ipython().system('pip install easyocr Pillow')

get_ipython().system('pip install easyocr jiwer opencv-python')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 751.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [6]:

import easyocr
import cv2
import pandas as pd
from shapely.geometry import Polygon
from difflib import SequenceMatcher
import os

def iou(boxA, boxB):
    polyA = Polygon([(boxA[0], boxA[1]), (boxA[2], boxA[3]), (boxA[4], boxA[5]), (boxA[6], boxA[7])])
    polyB = Polygon([(boxB[0], boxB[1]), (boxB[2], boxB[3]), (boxB[4], boxB[5]), (boxB[6], boxB[7])])
    if not polyA.is_valid or not polyB.is_valid:
        return 0
    intersection = polyA.intersection(polyB).area
    union = polyA.union(polyB).area
    return intersection / union if union != 0 else 0

def text_similarity(a, b):
    return SequenceMatcher(None, a.strip().lower(), b.strip().lower()).ratio()

reader = easyocr.Reader(['en'], gpu=False)
img_folder = "/content/img/img"
box_folder = "/content/box/box"

results = []

for img_file in sorted(os.listdir(img_folder)):
    if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    image_path = os.path.join(img_folder, img_file)
    base_name = os.path.splitext(img_file)[0]
    csv_path = os.path.join(box_folder, base_name + ".csv")

    if not os.path.exists(csv_path):
        print(f"No box file for {img_file}, skipping.")
        continue

    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    df = pd.read_csv(csv_path, header=None, engine='python', quoting=3, skip_blank_lines=True, on_bad_lines='skip')

    gt_boxes = df.iloc[:, :8].values
    gt_texts = df.iloc[:, 8].values

    detections = reader.readtext(image_path)

    for gt_box, gt_text in zip(gt_boxes, gt_texts):
        best_iou = 0
        best_text = ""
        for (bbox, text, conf) in detections:
            flat_bbox = [int(coord) for pt in bbox for coord in pt]

            flat_gtbox = list(gt_box.astype(int))
            if len(flat_bbox) == 8 and len(flat_gtbox) == 8:
                score = iou(flat_gtbox, flat_bbox)
                if score > best_iou:
                    best_iou = score
                    best_text = text
        sim = text_similarity(gt_text, best_text) if best_text else 0
        results.append((img_file, gt_text, best_text, best_iou, sim))

results_df = pd.DataFrame(results, columns=["image", "ground_truth", "predicted", "iou", "similarity"])
print(results_df.head())
print("Average Text Similarity:", results_df["similarity"].mean())


     image                    ground_truth                        predicted  \
0  000.jpg                   TAN WOON YANN                    tan woon yann   
1  000.jpg  BOOK TA .K(TAMAN DAYA) SDN BND  Book TA _k (TAMAN DAYA) SDN BHD   
2  000.jpg                        789417-W                         789117-W   
3  000.jpg                          JOHOR.                            JOHOR   
4  000.jpg        DOCUMENT NO : TD01167104                    Doctmerit Ijo   

        iou  similarity  
0  0.777551    1.000000  
1  0.605128    0.918033  
2  0.878049    0.875000  
3  0.641927    0.909091  
4  0.420967    0.432432  
Average Text Similarity: 0.7795372287057964
